In [1]:
# Load libraries
import numpy as np
import pandas as pd
import io

#  Mock Midterm Exercise

In this exercise you will have to implement code in the sections inside *Fill with Your Code* (*Load Data*, *Data Preprocessing* and *Create Model*) create a model to predict the column *exitus* in the dataset given. The dataset is already split into train, validation, and test subsets. To see to which subset belong each observation, you need to check the *dataset* column.

The code that is already written in this notebook **CANNOT BE CHANGED**. You can only add code in the *Fill with Your Code* section.

You must achieve in the last cell of this notebook an **AUC over test of at least 0.93**.

# Fill With Your Code

### [1] Load Data

In [13]:
dat = pd.read_csv('../data/dat.csv', sep = ",")

In [14]:
dat

,date,severity,mortality_ratio,age,num_proc,ambulatory,origin,expected_length,tip_grd,tip_adm,exitus,dataset
0,2016-01,NaN,0.408730,12596.0,21.0,0.0,NaN,151.0,Q,1.0,0,train
1,2016-01,NaN,0.306931,20973.0,22.0,NaN,NaN,99.0,Q,1.0,0,train
2,2016-01,4.0,0.278481,19611.0,19.0,NaN,NaN,87.0,NaN,1.0,0,train
3,2016-01,3.0,0.150289,13583.0,22.0,NaN,NaN,100.0,Q,NaN,0,train
4,2016-01,1.0,0.016573,18042.0,2.0,NaN,NaN,44.0,Q,1.0,0,train
...,...,...,...,...,...,...,...,...,...,...,...,...
32701,2016-12,2.0,0.028365,23619.0,2.0,NaN,NaN,2.0,NaN,1.0,0,test
32702,2016-12,1.0,0.000606,3935.0,1.0,NaN,1.0,2.0,M,1.0,0,test
32703,2016-12,NaN,0.040452,30163.0,4.0,NaN,NaN,2.0,M,NaN,0,test
32704,2016-12,NaN,0.000000,29012.0,4.0,NaN,NaN,0.0,NaN,1.0,0,test


In [15]:
dat['dataset'].value_counts()

dataset
train    22894
test      4907
val       4905
Name: count, dtype: int64

In [16]:
y = dat['exitus']
X = dat.drop(['exitus', 'date'], axis = 1)
X

,severity,mortality_ratio,age,num_proc,ambulatory,origin,expected_length,tip_grd,tip_adm,dataset
0,NaN,0.408730,12596.0,21.0,0.0,NaN,151.0,Q,1.0,train
1,NaN,0.306931,20973.0,22.0,NaN,NaN,99.0,Q,1.0,train
2,4.0,0.278481,19611.0,19.0,NaN,NaN,87.0,NaN,1.0,train
3,3.0,0.150289,13583.0,22.0,NaN,NaN,100.0,Q,NaN,train
4,1.0,0.016573,18042.0,2.0,NaN,NaN,44.0,Q,1.0,train
...,...,...,...,...,...,...,...,...,...,...
32701,2.0,0.028365,23619.0,2.0,NaN,NaN,2.0,NaN,1.0,test
32702,1.0,0.000606,3935.0,1.0,NaN,1.0,2.0,M,1.0,test
32703,NaN,0.040452,30163.0,4.0,NaN,NaN,2.0,M,NaN,test
32704,NaN,0.000000,29012.0,4.0,NaN,NaN,0.0,NaN,1.0,test


### [2]  Data Preprocessing

In [17]:
from sklearn.preprocessing import OneHotEncoder

categorical_vars = ['severity', 'origin', 'ambulatory', 'tip_grd', 'tip_adm']
non_categorical_vars = list(set(X.columns) - set(categorical_vars))
print(categorical_vars)
print(non_categorical_vars)

['severity', 'origin', 'ambulatory', 'tip_grd', 'tip_adm']
['dataset', 'num_proc', 'mortality_ratio', 'expected_length', 'age']


In [18]:
# 3 defining the model
ohe = OneHotEncoder(sparse_output = False)

# 4) Training model
ohe.fit(X)

# 5) Predicting
dat_ohe = pd.DataFrame(ohe.fit_transform(X[categorical_vars]))

# Optional
dat_ohe.columns = ohe.get_feature_names_out()

# Combine numerical and categorical
X_new = pd.concat((X[non_categorical_vars], dat_ohe), axis=1)

X_new

,dataset,num_proc,mortality_ratio,expected_length,age,severity_1.0,severity_2.0,severity_3.0,severity_4.0,severity_nan,...,ambulatory_0.0,ambulatory_1.0,ambulatory_nan,tip_grd_M,tip_grd_Q,tip_grd_nan,tip_adm_1.0,tip_adm_2.0,tip_adm_3.0,tip_adm_nan
0,train,21.0,0.408730,151.0,12596.0,0.0,0.0,0.0,0.0,1.0,...,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
1,train,22.0,0.306931,99.0,20973.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
2,train,19.0,0.278481,87.0,19611.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0
3,train,22.0,0.150289,100.0,13583.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
4,train,2.0,0.016573,44.0,18042.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32701,test,2.0,0.028365,2.0,23619.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0
32702,test,1.0,0.000606,2.0,3935.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
32703,test,4.0,0.040452,2.0,30163.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
32704,test,4.0,0.000000,0.0,29012.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0


In [19]:
X_new.isnull().any()

dataset            False
num_proc            True
mortality_ratio     True
expected_length     True
age                 True
severity_1.0       False
severity_2.0       False
severity_3.0       False
severity_4.0       False
severity_nan       False
origin_1.0         False
origin_2.0         False
origin_3.0         False
origin_4.0         False
origin_6.0         False
origin_8.0         False
origin_9.0         False
origin_nan         False
ambulatory_0.0     False
ambulatory_1.0     False
ambulatory_nan     False
tip_grd_M          False
tip_grd_Q          False
tip_grd_nan        False
tip_adm_1.0        False
tip_adm_2.0        False
tip_adm_3.0        False
tip_adm_nan        False
dtype: bool

In [20]:
### Fill missing values
from fancyimpute import IterativeImputer as MICE # pip install fancyimpute
numerical_vars = list(set(X_new.columns) - {'exitus', 'dataset'})
model = MICE()
X_new[numerical_vars] = model.fit_transform(X_new[numerical_vars])

### [3] Create Model

In [29]:
from sklearn.model_selection import train_test_split

X_train = X_new[dat['dataset'] == 'train'].drop('dataset', axis=1)
y_train = y[dat['dataset'] == 'train']

X_val = X_new[dat['dataset'] == 'val'].drop('dataset', axis=1)
y_val = y[dat['dataset'] == 'val']

X_test = X_new[dat['dataset'] == 'test'].drop('dataset', axis=1)
y_test = y[dat['dataset'] == 'test']

In [30]:
X_train

,num_proc,mortality_ratio,expected_length,age,severity_1.0,severity_2.0,severity_3.0,severity_4.0,severity_nan,origin_1.0,...,ambulatory_0.0,ambulatory_1.0,ambulatory_nan,tip_grd_M,tip_grd_Q,tip_grd_nan,tip_adm_1.0,tip_adm_2.0,tip_adm_3.0,tip_adm_nan
0,21.000000,0.408730,151.0,12596.0,0.0,0.0,0.0,0.0,1.0,0.0,...,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
1,22.000000,0.306931,99.0,20973.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
2,19.000000,0.278481,87.0,19611.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0
3,22.000000,0.150289,100.0,13583.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
4,2.000000,0.016573,44.0,18042.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22889,3.257889,0.000000,3.0,9157.0,1.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
22890,11.000000,0.011334,5.0,25533.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
22891,2.000000,0.000000,5.0,8537.0,1.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
22892,9.000000,0.030427,5.0,21855.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0


In [31]:
from sklearn.ensemble import RandomForestClassifier as model_constructor
from sklearn.metrics import roc_auc_score as metric

# Random Forest
n_estimators_values = [10, 100, 1000]
max_features_values = [2, 5, 10]
max_samples_values = [100, 1000, X_train.shape[0]]

params_grid = {'max_features': max_features_values,
              'n_estimators': n_estimators_values,
               'max_samples': max_samples_values}

num_iter = 1;
grid_results = pd.DataFrame(columns = ('max_features',
                                       'n_estimators',
                                       'max_samples',
                                       'metric_train',
                                       'metric_val'))

for max_features in params_grid['max_features']:
    for n_estimators in params_grid['n_estimators']:
        for max_samples in params_grid['max_samples']:

                        # Print trace
                        print('Iteracion = ' + str(num_iter))

                        # [3] Define model
                        model = model_constructor(max_features = max_features,
                                                  n_estimators = n_estimators,
                                                  max_samples = max_samples,
                                                  random_state = 0)

                        # [4] Train model
                        model.fit(X_train, y_train)


                        # [5] Predict
                        pred_train = model.predict(X_train) # predict!
                        pred_val = model.predict(X_val) # predict!

                        # [6] Compute metric
                        metric_train = metric(y_train, pred_train)
                        metric_val = metric(y_val, pred_val)

                        # print error
                        print('Metric train = %.2f - Metric validation = %.2f.'
                              % (metric_train, metric_val))

                        # Save iteration results
                        grid_results.loc[num_iter]=[ max_features,
                                                    n_estimators,
                                                    max_samples,
                                                 metric_train,
                                                 metric_val]
                        num_iter += 1




Iteracion = 1
Metric train = 0.50 - Metric validation = 0.50.
Iteracion = 2
Metric train = 0.54 - Metric validation = 0.53.
Iteracion = 3
Metric train = 0.94 - Metric validation = 0.56.
Iteracion = 4
Metric train = 0.50 - Metric validation = 0.50.
Iteracion = 5
Metric train = 0.52 - Metric validation = 0.51.
Iteracion = 6
Metric train = 1.00 - Metric validation = 0.57.
Iteracion = 7
Metric train = 0.50 - Metric validation = 0.50.
Iteracion = 8
Metric train = 0.52 - Metric validation = 0.51.
Iteracion = 9
Metric train = 1.00 - Metric validation = 0.57.
Iteracion = 10
Metric train = 0.54 - Metric validation = 0.54.
Iteracion = 11
Metric train = 0.55 - Metric validation = 0.54.
Iteracion = 12
Metric train = 0.94 - Metric validation = 0.57.
Iteracion = 13
Metric train = 0.50 - Metric validation = 0.50.
Iteracion = 14
Metric train = 0.54 - Metric validation = 0.53.
Iteracion = 15
Metric train = 1.00 - Metric validation = 0.58.
Iteracion = 16
Metric train = 0.50 - Metric validation = 0.50.
I

In [32]:
grid_results = grid_results.sort_values(by = ['metric_val', 'metric_train'], ascending = [False, True])
best_model = grid_results.iloc[0]
best_model

max_features       10.000000
n_estimators      100.000000
max_samples     22894.000000
metric_train        1.000000
metric_val          0.585164
Name: 24, dtype: float64

In [33]:
print('Old train data size = ' + str(X_train.shape))
print('Old train target size = ' + str(y_train.shape))

# Combine train and validación
X_train = np.concatenate((X_train, X_val), axis = 0)
y_train = np.concatenate((y_train, y_val), axis = 0)

print('New train data size = ' + str(X_train.shape))
print('New train target size = ' + str(y_train.shape))

Old train data size = (22894, 27)
Old train target size = (22894,)
New train data size = (27799, 27)
New train target size = (27799,)


In [34]:
# [3] Define model
model = model_constructor(criterion = 'gini',
                          max_depth = None,
                          min_samples_split = 2,
                          min_samples_leaf = 1,
                          max_features = int(best_model.max_features),
                          n_estimators =  int(best_model.n_estimators),
                          max_samples = int(best_model.max_samples),
                          random_state = 0) # Use same random_state as in training!!!

# Evaluate Model

In [38]:
# [4] Train model
model.fit(X_train, y_train)

RandomForestClassifier(max_features=10, max_samples=22894, random_state=0)

In [39]:
# [5] Predict
pred_train = model.predict(X_train)
pred_test = model.predict(X_test)
pred_val = model.predict(X_val)

J:\IE\SECOND TERM\MLII\venv\Lib\site-packages\sklearn\base.py:457: UserWarning: X has feature names, but RandomForestClassifier was fitted without feature names
  warnings.warn(
J:\IE\SECOND TERM\MLII\venv\Lib\site-packages\sklearn\base.py:457: UserWarning: X has feature names, but RandomForestClassifier was fitted without feature names
  warnings.warn(


In [40]:
# [6] Compute metric
metric_train = metric(y_train, pred_train[:,1])
metric_val = metric(y_test, pred_val[:,1])
metric_test = metric(y_val, pred_test[:,1])

IndexError: too many indices for array: array is 1-dimensional, but 2 were indexed

In [141]:
# print error
print('Metric train = %.2f - Metric val = %.2f - Metric test = %.2f'
      % (metric_train, metric_val, metric_test))

Metric train = 0.90 - Metric val = 0.60 - Metric test = 0.58
